In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import isnan, when, count, col
from pyspark.ml.evaluation import BinaryClassificationEvaluator

import sklearn 
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
spark = SparkSession.builder.appName('BreastCancer').getOrCreate()

# **Part 1 - Data Preprocessing**

In [ ]:
df = spark.read.csv('/FileStore/tables/breast_cancer_data.csv',inferSchema=True,header=True)

df.printSchema()

root
-- id: integer (nullable = true)
-- diagnosis: string (nullable = true)
-- radius_mean: double (nullable = true)
-- texture_mean: double (nullable = true)
-- perimeter_mean: double (nullable = true)
-- area_mean: double (nullable = true)
-- smoothness_mean: double (nullable = true)
-- compactness_mean: double (nullable = true)
-- concavity_mean: double (nullable = true)
-- concave points_mean: double (nullable = true)
-- symmetry_mean: double (nullable = true)
-- fractal_dimension_mean: double (nullable = true)
-- radius_se: double (nullable = true)
-- texture_se: double (nullable = true)
-- perimeter_se: double (nullable = true)
-- area_se: double (nullable = true)
-- smoothness_se: double (nullable = true)
-- compactness_se: double (nullable = true)
-- concavity_se: double (nullable = true)
-- concave points_se: double (nullable = true)
-- symmetry_se: double (nullable = true)
-- fractal_dimension_se: double (nullable = true)
-- radius_worst: double (nullable = true)
-- texture_worst: double (nullable = true)
-- perimeter_worst: double (nullable = true)
-- area_worst: double (nullable = true)
-- smoothness_worst: double (nullable = true)
-- compactness_worst: double (nullable = true)
-- concavity_worst: double (nullable = true)
-- concave points_worst: double (nullable = true)
-- symmetry_worst: double (nullable = true)
-- fractal_dimension_worst: double (nullable = true)
-- _c32: string (nullable = true)

**1.1 Cleaning Null Values**

In [ ]:
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show() #looks like _c32 column is null, we just won't include it in the features

+---+---------+-----------+------------+--------------+---------+---------------+----------------+--------------+-------------------+-------------+----------------------+---------+----------+------------+-------+-------------+--------------+------------+-----------------+-----------+--------------------+------------+-------------+---------------+----------+----------------+-----------------+---------------+--------------------+--------------+-----------------------+----+
 id|diagnosis|radius_mean|texture_mean|perimeter_mean|area_mean|smoothness_mean|compactness_mean|concavity_mean|concave points_mean|symmetry_mean|fractal_dimension_mean|radius_se|texture_se|perimeter_se|area_se|smoothness_se|compactness_se|concavity_se|concave points_se|symmetry_se|fractal_dimension_se|radius_worst|texture_worst|perimeter_worst|area_worst|smoothness_worst|compactness_worst|concavity_worst|concave points_worst|symmetry_worst|fractal_dimension_worst|_c32|
+---+---------+-----------+------------+--------------+---------+---------------+----------------+--------------+-------------------+-------------+----------------------+---------+----------+------------+-------+-------------+--------------+------------+-----------------+-----------+--------------------+------------+-------------+---------------+----------+----------------+-----------------+---------------+--------------------+--------------+-----------------------+----+
 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 569|
+---+---------+-----------+------------+--------------+---------+---------------+----------------+--------------+-------------------+-------------+----------------------+---------+----------+------------+-------+-------------+--------------+------------+-----------------+-----------+--------------------+------------+-------------+---------------+----------+----------------+-----------------+---------------+--------------------+--------------+-----------------------+----+

**1.2 Creating Features Column**

In [ ]:
assembler = VectorAssembler(inputCols=df.columns[2:-2],
                           outputCol='features') 
#so we're grabbing columns 2 to 31 which are our features. Column 1 is our label. Column 0 and 32 are not relevant.

output = assembler.transform(df) #applying our vector assembler to all our data

output.select('features').head(1) #notice features is a DenseVector containing all the features we combined

Out[5]: [Row(features=DenseVector([17.99, 10.38, 122.8, 1001.0, 0.1184, 0.2776, 0.3001, 0.1471, 0.2419, 0.0787, 1.095, 0.9053, 8.589, 153.4, 0.0064, 0.049, 0.0537, 0.0159, 0.03, 0.0062, 25.38, 17.33, 184.6, 2019.0, 0.1622, 0.6656, 0.7119, 0.2654, 0.4601]))]

**1.3 Encoding Label**

In [ ]:
output.groupBy('diagnosis').count().show() #so we got 2 categories for diagnosis

+---------+-----+
diagnosis|count|
+---------+-----+
 B| 357|
 M| 212|
+---------+-----+

In [ ]:
encoder = StringIndexer(inputCol="diagnosis", outputCol="diagnosis_cat")
encoded = encoder.fit(output).transform(output)
encoded.select('diagnosis_cat').show(5) #now M = 1, B = 0

+-------------+
diagnosis_cat|
+-------------+
 1.0|
 1.0|
 1.0|
 1.0|
 1.0|
+-------------+
only showing top 5 rows

**1.4 Creating Features and Label Dataframe**

In [ ]:
df_final = encoded.select('features', 'diagnosis_cat') #so the data we'll be using to apply Logistic Rregression will be the features and the label

df_final.show(3)

+--------------------+-------------+
 features|diagnosis_cat|
+--------------------+-------------+
[17.99,10.38,122....| 1.0|
[20.57,17.77,132....| 1.0|
[19.69,21.25,130....| 1.0|
+--------------------+-------------+
only showing top 3 rows

**1.5 Train Test Split**

In [ ]:
train, test = df_final.randomSplit([0.8,0.2]) #train will have 80%, test will have 20% of data

train.describe().show()

+-------+------------------+
summary| diagnosis_cat|
+-------+------------------+
 count| 465|
 mean|0.3741935483870968|
 stddev|0.4844351549535175|
 min| 0.0|
 max| 1.0|
+-------+------------------+

In [ ]:
test.describe().show()

+-------+-------------------+
summary| diagnosis_cat|
+-------+-------------------+
 count| 104|
 mean|0.36538461538461536|
 stddev|0.48386976318238856|
 min| 0.0|
 max| 1.0|
+-------+-------------------+

# **Part 2 - Creating our Model**

In [ ]:
logreg = LogisticRegression(featuresCol='features', labelCol='diagnosis_cat', predictionCol='prediction', maxIter=200)

logreg_model = logreg.fit(train) #fitting our train data

#**Part 3 - Evaluating Model**


In [ ]:
trainsummary = logreg_model.summary
trainsummary.predictions.describe().show()

+-------+------------------+------------------+
summary| diagnosis_cat| prediction|
+-------+------------------+------------------+
 count| 465| 465|
 mean|0.3741935483870968|0.3655913978494624|
 stddev|0.4844351549535175|0.4821142874181808|
 min| 0.0| 0.0|
 max| 1.0| 1.0|
+-------+------------------+------------------+

In [ ]:
pred = logreg_model.evaluate(test)
pred.predictions.show(5)

+--------------------+-------------+--------------------+--------------------+----------+
 features|diagnosis_cat| rawPrediction| probability|prediction|
+--------------------+-------------+--------------------+--------------------+----------+
[7.691,25.44,48.3...| 0.0|[14.8281641767721...|[0.99999963674636...| 0.0|
[8.597,18.6,54.09...| 0.0|[30.7610616438021...|[0.99999999999995...| 0.0|
[8.726,15.83,55.8...| 0.0|[7.64979688048313...|[0.99952408576950...| 0.0|
[8.888,14.64,58.7...| 0.0|[42.3118903066938...| [1.0,0.0]| 0.0|
[8.95,15.76,58.74...| 0.0|[36.5429628057369...|[0.99999999999999...| 0.0|
+--------------------+-------------+--------------------+--------------------+----------+
only showing top 5 rows

In [ ]:
y_true = pred.predictions.select(['diagnosis_cat']).collect()
y_pred = pred.predictions.select(['prediction']).collect()

print(classification_report(y_true, y_pred))

precision recall f1-score support

 0.0 0.96 0.98 0.97 66
 1.0 0.97 0.92 0.95 38

 accuracy 0.96 104
 macro avg 0.96 0.95 0.96 104
weighted avg 0.96 0.96 0.96 104

In [ ]:
print(confusion_matrix(y_true, y_pred))

[[65 1]
 [ 3 35]]